# French to English Translation with PyTorch

Building a sequence-to-sequence model from scratch using pure PyTorch.

This notebook demonstrates the complete pipeline:
1. Data acquisition and preprocessing
2. Vocabulary building  
3. Seq2Seq model architecture
4. Training and evaluation

All data is stored locally in `./data/` folder.

In [1]:
import torch 
import torch.nn as nn
import torch.optim as optim 
from torch.utils.data import Dataset, DataLoader 
from torch.nn.utils.rnn import pad_sequence

import pandas as pd
import numpy as np
from pathlib import Path
from collections import Counter
import random 
import re

random.seed(42)
np.random.seed(42)
torch.manual_seed(42)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"using device:{device}")

using device:cpu


## Step 1: Acquisition

Download and prepare data the giga-fren dataset(French - English parallel corpus)

In [2]:
data_path = Path('./data')
raw_path = data_path/'raw'
processed_path = data_path/'processed'

raw_path.mkdir(exist_ok=True)
processed_path.mkdir(exist_ok=True)

print(f"   Data: {data_path.absolute()}")
print(f"   Raw: {raw_path.absolute()}")
print(f"   Processed: {processed_path.absolute()}")


   Data: /home/hayk.minasyan/Project/translation/data
   Raw: /home/hayk.minasyan/Project/translation/data/raw
   Processed: /home/hayk.minasyan/Project/translation/data/processed


In [3]:
import os 

dataset_url = "https://s3.amazonaws.com/fast-ai-nlp/giga-fren.tgz"

dataset_file = data_path/"giga-fren.tgz"

if not dataset_file.exists():
    print(f"Downloading {dataset_url} ...")
    print("This will take a few minutes(2,4 GB) ... ")
    !wget {dataset_url} -O {dataset_file}
    print(f"Downloaded to {dataset_file}")
else:
    print(f"✅ Dataset already exists in {dataset_file}")

# Extract the dataset




✅ Dataset already exists in data/giga-fren.tgz


In [4]:
import tarfile 

en_file = raw_path/'giga-fren'/'giga-fren.release2.fixed.en'
fr_file = raw_path/'giga-fren'/'giga-fren.release2.fixed.fr'

if not en_file.exists() or not fr_file.exists():
    print("Extracting dataset...")
    with tarfile.open(dataset_file, 'r:gz') as tar:
        tar.extractall(path = raw_path)
    print(f"Extracted to {raw_path}")
else:
    print("Dataset already extracted")


print(f"\nFiles extracted:")
for f in (raw_path/'giga-fren').glob('*.fixed.*'):
    size_gb = f.stat().st_size / (1024**3)
    print(f"  {f.name}: {size_gb:.2f} GB")




Dataset already extracted

Files extracted:
  giga-fren.release2.fixed.en: 3.53 GB
  giga-fren.release2.fixed.fr: 4.25 GB


## Step 2: Data Exploration

Examine the raw dataset (22 million sentence pairs)

In [5]:
en_file = raw_path/'giga-fren'/'giga-fren.release2.fixed.en'
fr_file = raw_path/'giga-fren'/'giga-fren.release2.fixed.fr'

with open(en_file, 'r', encoding = 'utf-8') as f:
    en_count = sum(1 for _ in f)
with open(fr_file, 'r', encoding = 'utf-8') as f:
    fr_count = sum(1 for _ in f)

print("Dataset Statisitcs")
print(f"Enlglish sentences: {en_count:,}")
print(f"French sentences: {fr_count:,}")

Dataset Statisitcs
Enlglish sentences: 22,520,376
French sentences: 22,520,376


In [6]:
print(f"\n📝 Sample sentences:")
with open(en_file, 'r', encoding='utf-8') as en, \
     open(fr_file, 'r', encoding='utf-8') as fr:
    for i, (en_line, fr_line) in enumerate(zip(en, fr)):
        if i >= 5:
            break
        print(f"\n{i+1}.")
        print(f"  EN: {en_line.strip()}")
        print(f"  FR: {fr_line.strip()}")


📝 Sample sentences:

1.
  EN: Changing Lives | Changing Society | How It Works | Technology Drives Change Home | Concepts | Teachers | Search | Overview | Credits | HHCC Web | Reference | Feedback Virtual Museum of Canada Home Page
  FR: Il a transformé notre vie | Il a transformé la société | Son fonctionnement | La technologie, moteur du changement Accueil | Concepts | Enseignants | Recherche | Aperçu | Collaborateurs | Web HHCC | Ressources | Commentaires Musée virtuel du Canada

2.
  EN: Site map
  FR: Plan du site

3.
  EN: Feedback
  FR: Rétroaction

4.
  EN: Credits
  FR: Crédits

5.
  EN: Français
  FR: English


## Step 3: Data filtering

The full dataset is too large and contains mixed content. I'll extract only questions for a focused dataset.

- English: Questions starting with "Wh" (What, Where, When, Why, Which, Who)
- French: Sentences ending with "?"

In [7]:
re_eq = re.compile('^(Wh[^?.!]+\?)')
re_fq = re.compile('^([^?.!]+\?)') 

print("Filtering questions from 22M sentences ... ")

questions = []
count = 0

with open(en_file, 'r', encoding = 'utf-8') as en_f, \
    open(fr_file, 'r', encoding = 'utf-8') as fr_f:

    for en_line, fr_line in zip(en_f, fr_f):
        count += 1

        en_match = re_eq.search(en_line)
        fr_match = re_fq.search(fr_line)

        if en_match and fr_match:
            questions.append({
                'en': en_match.group().lower(),
                'fr': fr_match.group().lower()
            })

        if count % 1_000_000== 0:
            print(f"Processed: {count:,} | Found: {len(questions):,} questions")

print(f"Filtering complete!")
print(f"Total processed: {count:,}")
print(f"Questions found: {len(questions)}")
print(f"   Reduction: {count/len(questions):.1f}x smaller")



Filtering questions from 22M sentences ... 


Processed: 1,000,000 | Found: 3,481 questions
Processed: 2,000,000 | Found: 7,078 questions
Processed: 3,000,000 | Found: 9,095 questions
Processed: 4,000,000 | Found: 13,058 questions
Processed: 5,000,000 | Found: 16,997 questions
Processed: 6,000,000 | Found: 20,941 questions
Processed: 7,000,000 | Found: 22,480 questions
Processed: 8,000,000 | Found: 24,955 questions
Processed: 9,000,000 | Found: 27,302 questions
Processed: 10,000,000 | Found: 29,759 questions
Processed: 11,000,000 | Found: 32,019 questions
Processed: 12,000,000 | Found: 34,163 questions
Processed: 13,000,000 | Found: 36,305 questions
Processed: 14,000,000 | Found: 38,272 questions
Processed: 15,000,000 | Found: 40,282 questions
Processed: 16,000,000 | Found: 42,189 questions
Processed: 17,000,000 | Found: 44,179 questions
Processed: 18,000,000 | Found: 45,644 questions
Processed: 19,000,000 | Found: 46,975 questions
Processed: 20,000,000 | Found: 48,493 questions
Processed: 21,000,000 | Found: 49,841 questions
Proc

In [8]:
df = pd.DataFrame(questions)

csv_path = data_path/ 'questions.csv'
df.to_csv(csv_path, index = False)

print(F"Saved {len(df):,}questions to:")
print(f"Saved {len(df):,} questions to:")
print(f"   {csv_path.absolute()}")
print(f"\n📊 Dataset preview:")
print(df.head(10))
print(f"\n📈 Statistics:")
print(f"   French avg length: {df['fr'].str.split().str.len().mean():.1f} words")
print(f"   English avg length: {df['en'].str.split().str.len().mean():.1f} words")

Saved 52,331questions to:
Saved 52,331 questions to:
   /home/hayk.minasyan/Project/translation/data/questions.csv

📊 Dataset preview:
                                                  en  \
0                                    what is light ?   
1                                        who are we?   
2                            where did we come from?   
3                       what would we do without it?   
4  what is the absolute location (latitude and lo...   
5  what is the major aboriginal group on vancouve...   
6  what are the advantages and disadvantages of u...   
7  what types of land cover are associated with t...   
8                  what is the population of canada?   
9              which province is the most populated?   

                                                  fr  
0                          qu’est-ce que la lumière?  
1                                    où sommes-nous?  
2                                  d'où venons-nous?  
3                       que 

## Step 4: Vocabulary Building

Create word-to-index mappings for French and English.

In [9]:
def build_vocab(sentences, max_size = 10000, min_freq = 2):
    counter = Counter()
    for sent in sentences:
        counter.update(sent.split())

    vocab = ['<pad>', '<unk>', '<sos>', '<eos>']

    vocab += [word for word, count in counter.most_common(max_size)
    if count >= min_freq]

    #create mappings
    word2idx = {word: idx for idx, word in enumerate(vocab)}
    idx2word = {idx:word for idx, word in enumerate(vocab)}

    return vocab, word2idx, idx2word

print("Building vocabulary ...")
fr_vocab, fr_word2idx, fr_idx2word = build_vocab(df['fr'].tolist())
en_vocab, en_word2idx, en_idx2word = build_vocab(df['en'].tolist())

print(f"Vocab size:")
print(f"   French: {len(fr_vocab)}")
print(f"   English: {len(en_vocab)}")

print("Special tokens (indices):")
print(f"   <pad>: {fr_word2idx['<pad>']}")
print(f"   <unk>: {fr_word2idx['<unk>']}")
print(f"   <sos>: {fr_word2idx['<sos>']}")
print(f"   <eos>: {fr_word2idx['<eos>']}")

Building vocabulary ...


Vocab size:
   French: 10004
   English: 10004
Special tokens (indices):
   <pad>: 0
   <unk>: 1
   <sos>: 2
   <eos>: 3


In [10]:
import pickle

vocab_data = {
    'fr_word2idx': fr_word2idx,
    'en_word2idx': en_word2idx,
    'fr_idx2word': fr_idx2word,
    'en_idx2word': en_idx2word,
    'fr_vocab': fr_vocab,
    'en_vocab': en_vocab
}

vocab_path = processed_path/'vocab.pkl'
with open(vocab_path, 'wb') as f:
    pickle.dump(vocab_data, f)

print(f"Vocabularies saved to {vocab_path}")



Vocabularies saved to data/processed/vocab.pkl


## Load pre-trained fastText embeddings

In [13]:
import numpy as np

def load_vec_file(fname, max_vectors=200000):
    """Load fastText .vec file"""
    print(f"Loading {fname}...")
    embeddings = {}
    
    with open(fname, 'r', encoding='utf-8') as f:
        n, d = map(int, f.readline().split())
        print(f"  Total: {n:,} vectors, {d} dimensions")
        print(f"  Loading first {max_vectors:,}...")
        
        for i, line in enumerate(f):
            if i >= max_vectors:
                break
            values = line.rstrip().split(' ')
            word = values[0]
            vector = np.array(values[1:], dtype='float32')
            embeddings[word] = vector
            
            if (i + 1) % 50000 == 0:
                print(f"    Loaded {i+1:,}")
    
    print(f"  ✅ Loaded {len(embeddings):,} vectors")
    return embeddings

In [14]:
project_path = Path('.')  # Current directory (translation folder)
fr_vecs = load_vec_file(project_path / 'cc.fr.300.vec', max_vectors=200000)
en_vecs = load_vec_file(project_path / 'cc.en.300.vec', max_vectors=200000)

Loading cc.fr.300.vec...
  Total: 2,000,000 vectors, 300 dimensions
  Loading first 200,000...
    Loaded 50,000
    Loaded 100,000
    Loaded 150,000
    Loaded 200,000
  ✅ Loaded 200,000 vectors
Loading cc.en.300.vec...
  Total: 2,000,000 vectors, 300 dimensions
  Loading first 200,000...
    Loaded 50,000
    Loaded 100,000
    Loaded 150,000
    Loaded 200,000
  ✅ Loaded 200,000 vectors


In [15]:
def create_embedding_matrix(word2idx, vec_dict, emb_dim=300):
    """Create embedding matrix from pre-trained vectors"""
    vocab_size = len(word2idx)
    embedding_matrix = np.random.randn(vocab_size, emb_dim).astype('float32') * 0.01
    
    found = 0
    for word, idx in word2idx.items():
        if word in vec_dict:
            embedding_matrix[idx] = vec_dict[word]
            found += 1
    
    print(f"  Found {found}/{vocab_size} words ({found/vocab_size*100:.1f}%)")
    return embedding_matrix

print("Creating embedding matrices...")
fr_embedding_matrix = create_embedding_matrix(fr_word2idx, fr_vecs)
en_embedding_matrix = create_embedding_matrix(en_word2idx, en_vecs)

print(f"\n✅ Embedding matrices created:")
print(f"   French: {fr_embedding_matrix.shape}")
print(f"   English: {en_embedding_matrix.shape}")

# Save embeddings
embeddings_data = {
    'fr_embedding_matrix': fr_embedding_matrix,
    'en_embedding_matrix': en_embedding_matrix
    }

emb_path = processed_path / 'embeddings.pkl'
with open(emb_path, 'wb') as f:
    pickle.dump(embeddings_data, f)

print(f"✅ Saved embeddings to {emb_path}")

Creating embedding matrices...
  Found 6125/10004 words (61.2%)
  Found 6166/10004 words (61.6%)

✅ Embedding matrices created:
   French: (10004, 300)
   English: (10004, 300)
✅ Saved embeddings to data/processed/embeddings.pkl
